In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import json
tqdm.pandas()

In [ ]:
df1 = pd.read_csv("train.csv")
df1 = df1[["title","text","label"]]
df1.columns = ["heading","body","label"]
df1["label"] = df1["label"].progress_apply(lambda x: 1 if x == "clickbait" else 0)
df1 = df1.dropna()

In [ ]:
df2 = pd.read_csv("valid.csv")
df2 = df2[["title","text","label"]]
df2.columns = ["heading","body","label"]
df2["label"] = df2["label"].progress_apply(lambda x: 1 if x == "clickbait" else 0)
df2 = df2.dropna()

In [ ]:
bodies = pd.read_csv("fnc-1/train_bodies.csv")
headings = pd.read_csv("fnc-1/train_stances.csv")
headings["label"] = headings["Stance"].progress_apply(lambda x: 1 if x == "unrelated" else 0)
def get_body(x):
    for idx in range(len(bodies)):
        if bodies["Body ID"][idx] == x:
            return bodies["articleBody"][idx]
    return None
headings["body"]=headings["Body ID"].progress_apply(lambda x:get_body(x))
df3 = headings[["Headline","body","label"]]
df3.columns = ["heading","body","label"]
df3 = df3.dropna()

In [ ]:
bodies = pd.read_csv("fnc-1/competition_test_bodies.csv")
headings = pd.read_csv("fnc-1/competition_test_stances.csv")
headings["label"] = headings["Stance"].progress_apply(lambda x: 1 if x == "unrelated" else 0)
def get_body(x):
    for idx in range(len(bodies)):
        if bodies["Body ID"][idx] == x:
            return bodies["articleBody"][idx]
    return None
headings["body"]=headings["Body ID"].progress_apply(lambda x:get_body(x))
df4 = headings[["Headline","body","label"]]
df4.columns = ["heading","body","label"]
df4 = df4.dropna()

In [ ]:
with open("clickbait17-test-170720/truth.jsonl") as f:
    truths = [eval(i) for i in f.readlines()]
    truths = pd.DataFrame(truths)
with open("clickbait17-test-170720/instances.jsonl") as f:
    instances = [eval(i) for i in f.readlines()]
    instances = pd.DataFrame(instances)
def get_label(x):
    for idx in range(len(truths)):
        if truths["id"][idx] == x:
            if truths["truthClass"][idx]=="clickbait":
                return 1
            else:
                return 0
    return None
instances["label"] = instances["id"].progress_apply(lambda x: get_label(x))
instances["body"] = instances["targetParagraphs"].progress_apply(lambda x: " ".join(x))
instances["heading"] = instances["postText"].progress_apply(lambda x: " ".join(x))
df5 = instances[["heading","body","label"]]
df5 = df5.dropna()

In [ ]:
df = pd.concat([df1,df2,df3,df4,df5],ignore_index=True)

In [ ]:
df.to_csv("unclean/full.csv",index=False)

In [ ]:
df

In [ ]:
import numpy as np
train, validate, test = np.split(df.sample(frac=1, random_state=42), [int(.8*len(df)), int(.9*len(df))])

In [ ]:
train.to_csv("unclean/train.csv",index=False)

In [ ]:
validate.to_csv("unclean/val.csv",index=False)

In [ ]:
test.to_csv("unclean/test.csv",index=False)